In [960]:
#importing libraries

import random as rd
import hashlib
import phe
import rsa

In [961]:
#declaring of varibles
numberOfExperts = 3
numberOfCandidates = 5
#collection of homomophic public and private keys
dictionaryOfExpertPublicKeys = {}
dictionaryOfExpertPrivateKeys = {}

dictionaryOfCandidatesPublicKeys = {}
dictionaryOfCandidatesPrivateKeys = {}

#Variables for third framework@
dictionaryOfRandomNumberPerExpert = {}
dictionaryOftheSumOfRandomNumbermappedToEveryCandidate = {}
dictionaryOfRandomNumberGatheredFromOtherExperts = {}
dictionaryOfRandomNumberMappedToEveryCandidate = {}

#Collection of rsa public and private keys for digital signature
dictOfrsaPubKey = {}
dictOfrsaPriKey = {}
marksToCandidates = [[0 for i in range(numberOfCandidates)] for j in range(numberOfExperts)]


In [962]:
#this part of the code create the public and private homomorphic key pairs

for i in range(numberOfExperts):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfExpertPublicKeys[i] = x
    dictionaryOfExpertPrivateKeys[i] = y
    

for i in range(numberOfCandidates):
    x,y = phe.generate_paillier_keypair(n_length=128)
    dictionaryOfCandidatesPublicKeys[i]= x
    dictionaryOfCandidatesPrivateKeys[i] = y 

publicKeyEmployer,privateKeyEmployer = phe.generate_paillier_keypair(n_length=128)


In [963]:
#@code to generate rsa public and private keys
for i in range(numberOfExperts):
    (pubkey, privkey) = rsa.newkeys(512)
    dictOfrsaPubKey[i],dictOfrsaPriKey[i] = pubkey, privkey 


In [964]:
# function to create digital signature
def createDigitalSignature(message,i):
    privkey = dictOfrsaPriKey[i]
    temp = str(message).encode()
    signature = rsa.sign(temp, privkey, 'SHA-1')
    return signature

In [965]:
#this part of the code finds the highest marks gaining candidate id this acts as the candidate manager for the first frame work
def candidateManagerFrameWork1BestCandidate():
    array  = []
    for i in range(len(marksToCandidates[0])):
        sum = 0
        for j in range(len(marksToCandidates)):
            sum = sum + marksToCandidates[j][i][0]
        print(sum)
        array.append(sum)
        
    print("hihgest marks gaining candidate =",array.index( max(array)))    


In [966]:
#this part of the code defines the first frame work
def framreWork1():
    global marksToCandidates
    for i in range(numberOfExperts):
        for j in range(numberOfCandidates):
            mark = rd.randint(0,10)
            marksToCandidates[i][j] = [mark,createDigitalSignature(mark,i)]
    candidateManagerFrameWork1BestCandidate()

In [967]:
# framreWork1()
# print(marksToCandidates)

In [968]:
# tjhis part of the code works as the employer of the seconde frame work who finds the est candidate
def employerFindingBestCandidate(array):
    array = [privateKeyEmployer.raw_decrypt(x) for x in array]
    print(array.index(max(array)))
    print(array)

In [969]:
# this part of the code works as the candidate manager who adds the marks by multiplication of the encrytpion and than passes them to the employer
def candidateManagerFrameWork2BestCandidate():
    array  = []
    for i in range(len(marksToCandidates[0])):
        sum = 1
        for j in range(len(marksToCandidates)):
            sum = sum * marksToCandidates[j][i][0]
        array.append(sum)
    employerFindingBestCandidate(array)
    

In [970]:
def frameWork2():
    global marksToCandidates
    #this part of code simulates the expert giving marks to the candidates and storing them in form of matrix
    for i in range(numberOfExperts):
        for j in range(numberOfCandidates):
            mark = rd.randint(0,10)
            encryptedMark = publicKeyEmployer.raw_encrypt(mark)
            marksToCandidates[i][j] = [encryptedMark,createDigitalSignature(mark,i)]
    candidateManagerFrameWork2BestCandidate()
    

In [971]:
# frameWork2()


In [972]:
def generateRandomNumberForEveryExpert():
    # global dictionaryOfRandomNumberPerExpert
    generatedRandomNumbers = [rd.randint(0,100) for x in range(numberOfExperts)]
    # print(generatedRandomNumbers)
    return generatedRandomNumbers
    # print(dictionaryOfSumGeneratedPerRandomNumber)

In [973]:
def transferingTheRandomNumberOfEveryExpert():
        for i in range(numberOfExperts):
                dictionaryOfRandomNumberGatheredFromOtherExperts[i] = [dictionaryOfRandomNumberPerExpert[k][i] for k in range(numberOfExperts) if k != i]

In [974]:
def forwardTheRespectiveRandomNumbers(idR,generatedRandomNumbers):
    encryptedRandomNumber = dictionaryOfExpertPublicKeys[idR].raw_encrypt(generatedRandomNumbers[idR])
    message = encryptedRandomNumber
    return message
# ,createDigitalSignature(encryptedRandomNumber,idS)

In [975]:
# def expertsCalculatingTheHiddenScore(dictionaryOfRandomNumberMappedToEveryCandidate):
#     for i in range(numberOfCandidates):
#         for j in range(numberOfExperts):
#             mark = rd.randint(0,10)
#             hiddenScore = mark + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
#             sum = 0
#             for k in range(numberOfExperts):
#                     encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
#                     sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
#             hiddenScore = hiddenScore - sum
#             message = hiddenScore
#             marksToCandidates[j][i] = message
#             # ,createDigitalSignature(hiddenScore,j)

In [976]:
def performSMPCBetweenExperts():
    #this part of the code simulates the part where expert follow the smpc process
    #the random number assgned by all the experts has been mapped for every candidate
    
    for i in range(numberOfCandidates):
        matrixOfForwardedMessages = [[0 for i in range(numberOfExperts)] for j in range(numberOfExperts)]
        sumList = []
        for j in range(numberOfExperts):
            generatedRandomNumbers = generateRandomNumberForEveryExpert()
            sumOfTheRandomNumbers = sum(generatedRandomNumbers) 
            sumList.append(sumOfTheRandomNumbers)

            for k in range(numberOfExperts):
                if(j != k):
                    message = forwardTheRespectiveRandomNumbers(k,generatedRandomNumbers)
                    matrixOfForwardedMessages[j][k] = message
                else:
                    matrixOfForwardedMessages[j][k] = generatedRandomNumbers[j]
        dictionaryOfRandomNumberMappedToEveryCandidate[i] = matrixOfForwardedMessages
        dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i] = sumList
    print(dictionaryOfRandomNumberMappedToEveryCandidate)
    print(dictionaryOftheSumOfRandomNumbermappedToEveryCandidate)
    # print(dictionaryOfSumGeneratedPerRandomNumber)
    

    

In [977]:
def generateHiddenScoreFrameWork3():
    global marksToCandidates
    for i in range(numberOfCandidates):
        for j in range(numberOfExperts):
            mark = rd.randint(0,100)
            hiddenScore = mark + dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i][j]
            sum  = 0
            for k in range(numberOfExperts):
                if(j != k):
                    encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
                    # print(encryptedRandomNumber)
                    sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
                else: 
                    sum = sum + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
            hiddenScore  = hiddenScore -sum
            print(hiddenScore)
            marksToCandidates[j][i] = hiddenScore
            


In [978]:
def candidateManagerFrameWork3BestCandidate():
    array = []
    for i in range(numberOfCandidates):
        sum = 0
        for j in range(numberOfExperts):
            cummulativeScore = marksToCandidates[j][i]
            sum  = sum + cummulativeScore
        array.append(sum)
    print(array)
    print(array.index(max(array)))

In [979]:
def frameWork3():
    performSMPCBetweenExperts()
    generateHiddenScoreFrameWork3()
    candidateManagerFrameWork3BestCandidate()
    
        
    #     dictionaryOfRandomNumberMappedToEveryCandidate[i]   = matrixOfForwardedMessages  
    #     dictionaryOfSumGeneratedPerRandomNumber[i] = matrixOfTheSumOfRandomnumbers  
    # expertsCalculatingTheHiddenScore(dictionaryOfRandomNumberMappedToEveryCandidate)     
    # # candidateManagerFrameWork3BestCandidate()
    

In [980]:
frameWork3()

{0: [[49, 55154134435730401356657810331525163526520328902865775909917591491222495449415, 32064761905448534464425489774867650269861724988036905746933212597621010649641], [9067104569929787049968665245939349008247428712740525945049944776883187944990, 9, 41137135784413582778400364288118182360603568026465196595854569937981772393534], [7882870608751048109688480713842033841675491229425436020173699129860873673540, 51396204617736516883626740843842866026498685659318577464228285080628360800644, 82]], 1: [[35, 63478461898750251104020110581303663683558394746937725897741510424821174980093, 31665721065591026528794820213347625750897971114374874683675935928990915216151], [2005146545749649365677559134572494126210244340170968135537999236070447929491, 72, 36180695085633909169097180227124207368481779306378306317474758123846326749272], [22538579685979950890902018266316841122554544895042996772593263605457302235369, 11794275230808537565122238852453799064061592338276909937831534383618364468264, 66]], 2: [[17, 

In [981]:
def generateHiddenScoreFrameWork4():
    global marksToCandidates
    for i in range(numberOfCandidates):
        for j in range(numberOfExperts):
            mark = rd.randint(0,100)
            hiddenScore = mark + dictionaryOftheSumOfRandomNumbermappedToEveryCandidate[i][j]
            sum  = 0
            for k in range(numberOfExperts):
                if(j != k):
                    encryptedRandomNumber = dictionaryOfRandomNumberMappedToEveryCandidate[i][k][j]
                    # print(encryptedRandomNumber)
                    sum  = sum + dictionaryOfExpertPrivateKeys[j].raw_decrypt(encryptedRandomNumber)
                else: 
                    sum = sum + dictionaryOfRandomNumberMappedToEveryCandidate[i][j][j]
            hiddenScore  = hiddenScore -sum
            # print(hiddenScore)
            marksToCandidates[j][i] = publicKeyEmployer.raw_encrypt(hiddenScore)

In [982]:
def employerFindsTheBestCandidate(array):
    marksOfCandidates = [privateKeyEmployer.raw_decrypt(x) for x in array]
    print(marksOfCandidates)


In [983]:
def candidateManagerFrameWork4BestCandidate():
    array = []
    for i in range(numberOfCandidates):
        sum = 1
        for j in range(numberOfExperts):
            cummulativeScore = marksToCandidates[j][i]
            sum  = sum * cummulativeScore
        array.append(sum)
    employerFindsTheBestCandidate(array)


In [984]:
def framework4():
    performSMPCBetweenExperts()
    generateHiddenScoreFrameWork4()
    candidateManagerFrameWork4BestCandidate()


In [985]:
framework4()

{0: [[50, 34569766656311595518741790489861219386867250920061909617012717482414877928135, 20822800823060078979618477409910220706947319502315554972787880640613760577668], [29406780470501716036205243471996299804731183955493181291765819974944985679734, 88, 18574230304204178043792514790219153095105473849263316761588859112901025982443], [19362817034232730932700991019341091758483827315025493754924528672618809136311, 11200960760468460312455250258613063962574240029964805262457608099139636735491, 21]], 1: [[64, 23814014677028035466396433402725564806595224443141892854495361596972737646963, 20006645951304741169139733076175851736250717626482542285938694532203641759483], [34633855500003485112110324945841012772337228688129942870748915002500079753588, 66, 29841077765067416189053114493492594517108831674787064631185910232093593370614], [35865478531315509082869757911181119414770096077494648611082281935123431021174, 7103678521457830877639928478559323577229026208767939001456209077551769899282, 36]], 2: [[4